# Estimation problem workflow

For a multivariate random variable $X$ and a real-valued function $f$ the goal is to compute 

$$
    \mathbb{E}[f(X)], \text{ where } f: \mathbb{R}^n \mapsto I \subset \mathbb R
$$

Here, this is achieved using quantum amplitude estimation (QAE), based on a $\mathcal{A}$ operator in the format

$$
    \mathcal{A}|0\rangle = \sqrt{1 - a}|\Psi_0\rangle + \sqrt{a}|\Psi_1\rangle
      = \sum_{\hat x = 0}^{2^n - 1} \sqrt{p_{\phi(\hat x)} (1 - \hat f(\phi(\hat x)))} |x\rangle |0\rangle
      + \sum_{\hat x = 0}^{2^n - 1} \sqrt{p_{\phi(\hat x)} \hat f(\phi(\hat x))} |x\rangle |1\rangle
$$

To revert the scalings from $f$ to $\hat f$ and possible post-processing from the function approximation, the result of QAE, $\tilde a$, is mapped to the result via a post-processing function $\eta$

$$
\mathbb{E}[f(X)] = \eta(a) \approx \eta(\tilde a)
$$

**Notes:**

An amplitude function $F$ of a function $\hat f$ is a mapping

$$
F: |x\rangle|0\rangle \mapsto \sqrt{1 - \hat f(x)}|x\rangle|0\rangle + \sqrt{\hat f(x)}|x\rangle|1\rangle
$$

where

$$
\hat f: \mathbb{N}_0 \rightarrow [0, 1].
$$

If we have a function $f$ that does not act on these spaces we must normalize the image and apply an affine transformation from the input domain to $\mathbb{N}_0$. Let the function $f$ be defined on $2^n$ equidistant points in $[a, b]$:

$$
f: [a, b] \rightarrow [c, d]
$$

then the rescaled version is

$$
\hat f(x) = \frac{f(\phi(x)) - c}{d - c}
$$

where $\phi(x) = a + (b - a) x / 2^n$ is the affine transformation.

The normalized function $\hat f$ can be implemented in different manners. In general this requires a post-processing step, $\zeta$:

$$
\hat f \approx \zeta(T\hat f)
$$

where $T\hat f$ is the amplitude we can map onto the qubits.
One example is a Taylor approximation of $\sin^2$ which can be mapped to qubits using RY gates.

Examples:

* quadratic terms can be approximated with
    $$
    ax^2 \approx \sin^2(c\sqrt{a}x) / c^2
    $$
    where the sine part is estimated with amplitude estimation and the rescaling is $\zeta(x) = x/c^2$
    
* linear terms can be approximated as
    $$
    ax \approx \zeta\left( \sin^2\left(\frac{\pi}{4} + \frac{\pi c}{2} \left(f(x) - \frac{1}{2}\right)\right) \right) 
    $$
    with
    $$
    \zeta(x) = \frac{2}{\pi c}\left(x - \frac{1}{2}\right) + \frac{1}{2}
    $$

Let $\tilde a$ be the output of amplitude estimation, then in general we have to apply a post-processing in the form of

$$
c + (d - c) \zeta(\tilde a)
$$

## Defining the random variable

The distribution of the random variable is loaded from the circuit library. The function will be evaluated on the x-values ``[0, ..., 2 ** num_qubits - 1]``. 

In [ ]:
from qiskit.circuit.library import NormalDistribution
X = NormalDistribution(num_qubits=3, mu=1, sigma=0.5, bounds=(0, 2))

## Defining the objective function

### Workflow #1

Similar to the oracle compiler: take a Python function and synthesize the circuit automatically.

In [ ]:
def classical_f(x):
    if x > 1:
        return x ** 2
    return x


from qiskit.circuit.library import AmplitudeFunction
f = AmplitudeFunction(classical_f, domain=(0, 2))  # domain must be the same as the bounds of the probability dist
post_processing = f.post_processing  # c + (d - c) zeta

### Workflow #2

Similar to the `UnivariatePiecewiseLinearObjective`, specify the function and construct the circuit from this information. Includes rescalings.

In [ ]:
from qiskit.circuit.library import LinearAmplitudeFunction
f = LinearAmplitudeFunction(num_state_qubits=3, 
                            slope=[-1, 1],
                            offset=[1, 0],
                            breakpoints=[0, 1],
                            domain=(0, 2))

### Workflow #3

Manual.

In [ ]:
from qiskit.circuit.library import PiecewiseLinearPauliRotations

slopes = np.array([-1, 1])
offsets = np.array([1, 0])
breakpoints = np.array([0, 1])
domain = (0, 2)
num_state_qubits = 3
N = 2 ** num_state_qubits

# apply rescaling to the right domain 
phi_inv = lambda x: (x - domain[0]) / (domain[1] - domain[0]) * N
breakpoints = phi_inv(breakpoints)
slopes = (domain[1] - domain[0]) / (N - 1) * slopes
# offsets remain the same

# apply normalization of the image space
offsets = (offsets - fmin) / (fmax - fmin)
slopes = slopes / (fmax - fmin)

# apply taylor 
offsets *= c * np.pi / 2
slopes *= c * np.pi / 2

f = PiecewiseLinearPauliRotations(num_state_qubits, 
                                  breakpoints=breakpoints,
                                  slopes=2 * slopes,  
                                  offsets=2 * offsets)  

### Conclusion

Option #2 makes sense for now. This means porting most of the `UnivariatePiecewiseLinearObjective` to the circuit library.

## Defining the A operator

### Workflow #1

Stack together the function circuit and probability distribution circuit.

In [ ]:
from qiskit.circuit import QuantumCircuit

A = QuantumCircuit(f.num_qubits)
A.compose(X, inplace=True)
A.compose(f, inplace=True)

### Workflow #2

Can introduce a class for that but I don't really think the work it does justifies a class, see rather the section about encapsulating the entire workflow.

In [ ]:
from qiskit.circuit.library import EstimationProblem  # or aqua algorithms

A = EstimationProblem(f, X)  # but here f is not the circuit but a function! and we do all the synthesisa

## Run QAE

QAE in the basic form takes the $\mathcal{A}$ operator and a post processing function.

In [ ]:
from qiskit import Aer
from qiskit.aqua.algorithms import AmplitudeEstimation

ae = AmplitudeEstimation(num_eval_qubits=4, state_in=A, post_processing=f.post_processing)
result = ae.run(Aer.get_backend('qasm_simulator'))

## Encapsulating the workflow

Add a class that is fully equivalent to `UnivariatePiecewiseLineaObjective`? Essentially this means accepting the probabilit